In [1]:
from planner2 import load_scene_graph, generate_prompt, get_navigation_plan
from robots import robots

In [2]:
scene_graph = load_scene_graph("/Users/nayunkim/Documents/GitHub/repairPlan/data/3D_Scene_Graph_large.json")
prompt = generate_prompt(scene_graph, robots)

In [5]:
user_input = "there is a stain on the ceiling in room id 309. the robot is currently in room id 314"
navigation_plan = get_navigation_plan(user_input, scene_graph, robots)

In [6]:
print(navigation_plan)

1. **Identify Defect:**
   - Reasoning: The defect described is a stain on the ceiling in room ID 309. In the 3D scene graph, the ceiling in room 309 is represented by the element with ID 698. This is identified as the target defect node based on the description provided.

2. **Check Reachability:**
   - Reasoning: The ceiling in room 309 is at a height of 3.7 meters. We need to compare this height with each robot's `max_reach_height` to determine which robots can reach the defect.
     - **robot1**: Max reach height is 2.5 meters. Not reachable.
     - **robot2**: Max reach height is 3.0 meters. Not reachable.
     - **robot3**: Max reach height is 3.5 meters. Not reachable.
     - **robot4**: Max reach height is 3.7 meters. Reachable.
     - **robot5**: Max reach height is 2.8 meters. Not reachable.
     - **robot6**: Max reach height is 2.3 meters. Not reachable.
     - **robot7**: Max reach height is 2.2 meters. Not reachable.
     - **robot8**: Max reach height is 2.6 meters. Not 